In [5]:
import json # will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
from sklearn.ensemble import RandomForestClassifier # for training the algorithm
from sklearn.ensemble import ExtraTreesClassifier # for training the algorithm
import joblib # for saving algorithm and preprocessing objects

In [6]:
df = pd.read_csv('./data_.csv', skipinitialspace=True)
x_cols = [c for c in df.columns if c != 'income']
# set input matrix and target column
X = df[x_cols]
y = df['income']
# show first rows of data
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1234)

In [4]:
train_mode = dict(X_train.mode().iloc[0])
X_train = X_train.fillna(train_mode)
print(train_mode)

{'age': 31.0, 'workclass': 'Private', 'fnlwgt': 121124, 'education': 'HS-grad', 'education-num': 9.0, 'marital-status': 'Married-civ-spouse', 'occupation': 'Prof-specialty', 'relationship': 'Husband', 'race': 'White', 'sex': 'Male', 'capital-gain': 0.0, 'capital-loss': 0.0, 'hours-per-week': 40.0, 'native-country': 'United-States'}


In [16]:
# convert categoricals
encoders = {}
for column in ['workclass', 'education', 'marital-status',
                'occupation', 'relationship', 'race',
                'sex','native-country']:
    categorical_convert = LabelEncoder()
    X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column] = categorical_convert

In [17]:
# train the Random Forest algorithm
rf = RandomForestClassifier(n_estimators = 100)
rf = rf.fit(X_train, y_train)

In [18]:
# train the Extra Trees algorithm
et = ExtraTreesClassifier(n_estimators = 100)
et = et.fit(X_train, y_train)

In [19]:
# save preprocessing objects and RF algorithm
joblib.dump(train_mode, "./train_mode.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(rf, "./random_forest.joblib", compress=True)
joblib.dump(et, "./extra_trees.joblib", compress=True)

['./extra_trees.joblib']

In [23]:
et

ExtraTreesClassifier()

## Test

In [10]:
import requests

In [13]:
for i in range(100):
    input_data = dict(X_test.iloc[i])
    target = y_test.iloc[i]
    r = requests.post("http://127.0.0.1:8001/api/v1/income_classifier/predict?status=ab_testing", input_data)
    response = r.json()
    print(response)
    # provide feedback
    requests.put("http://127.0.0.1:8001/api/v1/mlrequests/{}".format(response["request_id"]), {"feedback": target})

{'probability': 0.62, 'label': '>50K', 'status': 'OK', 'request_id': 102}
{'probability': 0.38, 'label': '<=50K', 'status': 'OK', 'request_id': 103}
{'probability': 0.06, 'label': '<=50K', 'status': 'OK', 'request_id': 104}
{'probability': 0.77, 'label': '>50K', 'status': 'OK', 'request_id': 105}
{'probability': 0.0, 'label': '<=50K', 'status': 'OK', 'request_id': 106}
{'probability': 0.0, 'label': '<=50K', 'status': 'OK', 'request_id': 107}
{'probability': 0.0, 'label': '<=50K', 'status': 'OK', 'request_id': 108}
{'probability': 0.21, 'label': '<=50K', 'status': 'OK', 'request_id': 109}
{'probability': 0.0, 'label': '<=50K', 'status': 'OK', 'request_id': 110}
{'probability': 0.51, 'label': '>50K', 'status': 'OK', 'request_id': 111}
{'probability': 0.72, 'label': '>50K', 'status': 'OK', 'request_id': 112}
{'probability': 0.04, 'label': '<=50K', 'status': 'OK', 'request_id': 113}
{'probability': 0.15, 'label': '<=50K', 'status': 'OK', 'request_id': 114}
{'probability': 0.01, 'label': '<